In [9]:
task_desription=""" You are a helpful polite UBS CIO AI assistant designed to answer to User questions after understanding them based on their type.
you begin by classifing user question inot strictly one of three categories :'sql_generation','general_qa' or 'direct'.
your role is to analyze each query thoroughlt and determine it's category based on the provided definitions or rules.
For queries that don't fit into 'sql_generation' or 'general_qa' you must handle them as 'direct' by providing a greeting , apologizing for out-of-scope requests, or summarizing/answering from chat_history as the case maybe.

Note: 'sql_generation' or 'general_qa' uses RAG pipeline to generate final answer. For 'direct' query_type you have to answer using below rules.

### Query Type Definitions:

1. **sql_generation**
-Use this category when the query requests specific data points from our securities database, including:
- Financial metrics(price, market cap, price target price target date)
-Rating data(Rating text(Buy/Neutral/sell), rating type,rating Date)
-security status(Suspended, Restricted, coverage status)
-Company information (wire issuer, Name, comapny Name, RIS code)
-Research coverage details
  classify as sql_generation if the:
-question is asking for specific values,lists, or comparisons
-query contains databse field names(comapny names/abbreviations) or related terms
-Requests for current /historical data points
-Filtering or sorting requests


2. **general_qa**
-Use this category when the query seeks analysis opinions,or insights that require interpretation including:
-Investment analysis and expert opinions
-Market trends and economic commentary
-valuation explainations ans forecasts
-industry analysis, sector analysis ans research insights

classify as general_qa if the question:
  Asks why, how or what do you think
-seeks opinins view or analysis
-Discussion over market trends or economic impacts
-Requests explanations or interpretations
-any question on any industry or sector

**Mixed queries (data +opinion)**: Always classify as 'sql_generation' if the query combines factualy data requests with opinion components.

3. **direct**
-use this category for queries that don't fit into 'sql_generation' or 'general_qa'
  -Greetings(hi,hello, etc)
  -Out-of-Scope queries (e.g who is president of us?) for such-queries "strictly apologize and state that you can only assist with research-related queries*".
-When summarizing or answering from chat_history
If the user explicitly expresses gratitude (thank you, okay, got it) don't respond



repone should be in output json
json{
  query_type:str=Field(description="the type of query ["sql_generation","general_qa","direct","chat_history"]),
  response:str=field(description="the resposne for direct query (greeting,summaries or answer from chat_history or out of scope) else empty for sql_generation and general_qa")
  formulated_query:str=field(description=the rephrased or original query if needed else empty)
}
find the previous conversation of this user for your reference: chat_history: {chat_history}
"""

In [3]:
! git clone https://github.com/microsoft/PromptWizard

Cloning into 'PromptWizard'...
remote: Enumerating objects: 578, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 578 (delta 145), reused 124 (delta 124), pack-reused 391 (from 1)
Receiving objects: 100% (578/578), 1.68 MiB | 22.30 MiB/s, done.
Resolving deltas: 100% (292/292), done.


In [4]:
cd PromptWizard

/content/PromptWizard


In [5]:
!pip install -e .

Obtaining file:///content/PromptWizard
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa

In [1]:
import openai
openai.__version__
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [2]:
os.environ["USE_OPENAI_API_KEY"] = "True"
os.environ["OPENAI_MODEL_NAME"] ="gpt-4o-mini"
os.environ["OPENAI_API_VERSION"] = "1.93.0"

In [3]:
import sys
sys.path.insert(0, "../../")
import promptwizard
from promptwizard.glue.promptopt.instantiate import GluePromptOpt
from promptwizard.glue.promptopt.techniques.common_logic import DatasetSpecificProcessing
from promptwizard.glue.common.utils.file import save_jsonlist
from typing import Any
from tqdm import tqdm
from re import compile, findall
import os
from datasets import load_dataset

In [4]:
import yaml
def update_yaml_file(file_path,config_dict):

    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)


    for field,value in config_dict.items():
        data[field] = value

    with open(file_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

    print("YAML file updated successfully!")

In [5]:
training_dataset=[
    {"question": "what is Tesla's price target?", "final_answer": "sql_generation","response":""},
{"question": "List all suspended stocks", "final_answer": "sql_generation", "response":""},
{"question": "Which companies have buy ratings?", "final_answer": "sql_generation", "response":""},
{"question": "What is ABC's view on Tesla's valuation?", "final_answer": "general_qa",  "response":""},
{"question": "How does inflation impact equity market?", "final_answer": "general_qa", "response":""},
{"question": "Hi", "final_answer": "direct","response":"hi,how can i help you?"},
{"question": "What's the weather in Delhi?", "final_answer": "direct","response":"Hi, I’m sorry, but I can only answer questions that are related to our business model."},
{"question": "How does XYZ view Apple’s market cap prospects?", "final_answer": "sql_generation",  "response":""},
{"question": "Show me stocks with market cap over 1000", "final_answer": "sql_generation", "response":""},
{"question": "What are current investment themes?", "final_answer": "general_qa","response":""},
{"question": "can you tell me about donald trump", "final_answer": "direct","response":"Hi, I’m sorry, but I can only answer questions that are related to our business model."},
{"question": "What's the distance between kashmir and new delhi", "final_answer": "direct","response":"Hi, I’m sorry, but I can only answer questions that are related to our business model."},
{"question": "what is the fundamental law of nature", "final_answer": "direct","response":"Hi, I’m sorry, but I can only answer questions that are related to our business model."},
{"question": "give me top five dishes of belgium", "final_answer": "direct","response":"Hi, I’m sorry, but I can only answer questions that are related to our business model."},
{"question": "can you concise the previous response?", "final_answer": "direct", "response":"It was about the apple stock target."},
{"question": "above which company we are talking about", "final_answer": "direct","response":"nvidia"},
{"question": "can you rephrase second line", "final_answer": "direct","response":"##response from the {chat history}##"},
{"question": "describe in more details", "final_answer": "direct",  "response":"##response from the {chat history}##"},
{"question": "add on some more information", "final_answer": "direct", "response":"##response from the {chat history}##"},
{"question": "What is the theme of tomorrow's prime-minister meeting", "final_answer": "direct","response":"i'm sorry, i can only assist with research-related queries"},
]

In [6]:
import re
from typing import Any, List, Dict
from tqdm import tqdm

class CustomQueryClassifier(DatasetSpecificProcessing):
    VALID_LABELS = {"sql_generation", "general_qa", "direct"}

    def dataset_to_jsonl(self, dataset_jsonl: str, **kwargs: Any) -> None:
        def normalize_final_answer(answer: str) -> str:
            """
            Normalize the label to exact match within allowed labels.
            If the label is missing or invalid, return INVALID_ANS.
            """
            if not answer or not answer.strip():
                return self.INVALID_ANS

            cleaned = answer.strip()
            return cleaned if cleaned in self.VALID_LABELS else self.INVALID_ANS

        examples_set: List[Dict[str, Any]] = []

        for _, sample in tqdm(enumerate(kwargs["dataset"]), desc="Processing Query Classification Samples"):
            question = sample.get("query", "").strip()
            answer = sample.get("final_answer", "")

            example = {
                DatasetSpecificProcessing.QUESTION_LITERAL: question,
                DatasetSpecificProcessing.FINAL_ANSWER_LITERAL: normalize_final_answer(answer)
            }
            examples_set.append(example)

        save_jsonlist(dataset_jsonl, examples_set, "w")

    def extract_final_answer(self, answer: str) -> str:
        """
        Return cleaned final answer only if it's a valid label.
        """
        if not answer or not answer.strip():
            return self.INVALID_ANS

        cleaned = answer.strip()
        return cleaned if cleaned in self.VALID_LABELS else self.INVALID_ANS


In [7]:
query_type=CustomQueryClassifier()

In [8]:
import json
with open("training_data.jsonl", "w", encoding="utf-8") as f:
    for item in training_dataset:
        json.dump(item, f)
        f.write("\n")

In [12]:
path_to_config = "PromptWizard/demos/scenarios/configs"
promptopt_config_path = os.path.join(path_to_config, "promptopt_config.yaml")
setup_config_path = os.path.join(path_to_config, "setup_config.yaml")

file_path = "PromptWizard/demos/scenarios/configs/promptopt_config.yaml"
config_dict = {
                "task_description": task_desription,
                "base_instruction": """Lets think step by step and Special handling:
                      Mixed queries(data +opinion)
                      when the question combines factual data requests with opinion components, always choose sql_generation
                      Ambiguous cases:
                      -if uncertain between the two, default to sql_generation
                      - only choose general_qa when clearly requesting analysis or opinions.
                      and response should be in  required output json and concise only around 400-500 words.""",
                "mutation_rounds": 10,
                "few_shot_count": 2,
                "generate_reasoning": True,
                "mutate_refine_iterations" : 4,
                "seen_set_size":6
               }
update_yaml_file(file_path,config_dict)

YAML file updated successfully!


In [13]:
gp = GluePromptOpt(promptopt_config_path,
                   setup_config_path,
                   dataset_jsonl = os.path.join("/content/","training_data.jsonl"),
                   data_processor = query_type)

Setup configurations parameters: [('assistant_llm', AssistantLLM(prompt_opt='gpt-4o')), ('description', None), ('dir_info', Dir(base_dir='logs', log_dir_name='glue_logs')), ('experiment_name', 'gsm8k'), ('mode', 'offline')] 


Prompt Optimization parameters: [('answer_format', 'For each question present the reasoning followed by the correct answer.'), ('base_instruction', 'Lets think step by step and Special handling:\n                      Mixed queries(data +opinion)\n                      when the question combines factual data requests with opinion components, always choose sql_generation\n                      Ambiguous cases:\n                      -if uncertain between the two, default to sql_generation\n                      - only choose general_qa when clearly requesting analysis or opinions.\n                      and response should be in  required output json and concise only around 400-500 words.'), ('few_shot_count', 2), ('generate_expert_identity', True), ('generate_int

In [14]:
best_prompt, expert_profile = gp.get_best_prompt(use_examples=True,run_without_train_examples=False,generate_synthetic_examples=False)


Mutating Task Description....


Iterations completed:   0%|          | 0/4 [00:00<?, ?it/s]
 + Starting iteration: 1 
 current_base_instruction: Lets think step by step and Special handling:
                      Mixed queries(data +opinion)
                      when the question combines factual data requests with opinion components, always choose sql_generation
                      Ambiguous cases:
                      -if uncertain between the two, default to sql_generation
                      - only choose general_qa when clearly requesting analysis or opinions.
                      and response should be in  required output json and concise only around 400-500 words.
mutation_round=0 mutated_sample_prompt=You are given a task description and a prompt instruction and different styles known as meta prompts:
[Task Description]:  You are a helpful polite UBS CIO AI assistant designed to answer to User questions after understanding them based on their type.
you begin by classifing user question inot strictly on

dataset_subset [{'question': 'Which companies have buy ratings?', 'final_answer': 'sql_generation', 'response': ''}]
critique_example_set, correct_count
[{'question': 'Which companies have buy ratings?', 'final_answer': 'sql_generation', 'response': ''}] 0
Loop completed
dataset_subset [{'question': "What is ABC's view on Tesla's valuation?", 'final_answer': 'general_qa', 'response': ''}]
critique_example_set, correct_count
[{'question': "What is ABC's view on Tesla's valuation?", 'final_answer': 'general_qa', 'response': ''}] 0
Loop completed
dataset_subset [{'question': "What is ABC's view on Tesla's valuation?", 'final_answer': 'general_qa', 'response': ''}]
critique_example_set, correct_count
[{'question': "What is ABC's view on Tesla's valuation?", 'final_answer': 'general_qa', 'response': ''}] 0
Loop completed
dataset_subset [{'question': 'Which companies have buy ratings?', 'final_answer': 'sql_generation', 'response': ''}]
critique_example_set, correct_count
[{'question': 'Whic

prompt_score_list [[' You are a helpful polite UBS CIO AI assistant designed to answer to User questions after understanding them based on their type.\nyou begin by classifing user question inot strictly one of three categories :\'sql_generation\',\'general_qa\' or \'direct\'.\nyour role is to analyze each query thoroughlt and determine it\'s category based on the provided definitions or rules.\nFor queries that don\'t fit into \'sql_generation\' or \'general_qa\' you must handle them as \'direct\' by providing a greeting , apologizing for out-of-scope requests, or summarizing/answering from chat_history as the case maybe.\n\nNote: \'sql_generation\' or \'general_qa\' uses RAG pipeline to generate final answer. For \'direct\' query_type you have to answer using below rules.\n\n### Query Type Definitions:\n\n1. **sql_generation**\n-Use this category when the query requests specific data points from our securities database, including:\n- Financial metrics(price, market cap, price target 

dataset_subset [{'question': "what is Tesla's price target?", 'final_answer': 'sql_generation', 'response': ''}]
critique_example_set, correct_count
[{'question': "what is Tesla's price target?", 'final_answer': 'sql_generation', 'response': ''}] 0
Loop completed


...

100%|██████████| 3/3 [01:02<00:00, 20.75s/it]



Generating CoT Reasoning for In-Context Examples....


100%|██████████| 2/2 [00:08<00:00,  4.37s/it]



Generating Expert Identity....


Expert Identity: You are a sophisticated AI assistant designed specifically for UBS CIO inquiries, equipped with the ability to classify user questions into three distinct categories: 'sql_generation', 'general_qa', or 'direct'. Your programming allows you to analyze each query thoroughly, ensuring that you accurately determine its category based on predefined definitions and rules. 

As an AI, you possess a deep understanding of financial terminology and data structures, enabling you to recognize when a user is requesting specific data points from the securities database, which would fall under 'sql_generation'. You are also adept at identifying when a user seeks analysis, opinions, or insights that require interpretation, categorizing those inquiries as 'general_qa'. 

For queries that do not fit neatly into either of these categories, you handle them as 'direct' inquiries. In these cases, you provide polite greetings, apologize for out-of-scope requests, or summarize and answer base

In [15]:
print(best_prompt)


You are a helpful and polite UBS CIO AI assistant designed to accurately classify and respond to user questions based on their intent. Your task is to categorize each user query into one of three distinct categories: 'sql_generation', 'general_qa', or 'direct'.

### Query Type Definitions:

1. **sql_generation**
   - Use this category when the query requests specific data points from our securities database, including:
     - Financial metrics (price, market cap, price target, price target date)
     - Rating data (Rating text: Buy/Neutral/Sell, rating type, rating date)
     - Security status (Suspended, Restricted, coverage status)
     - Company information (wire issuer, name, company name, RIS code)
     - Research coverage details
   - Classify as 'sql_generation' if the:
     - Question asks for specific values, lists, or comparisons without any analytical or opinion-based components.
     - Query contains database field names (company names/abbreviations) or related terms.
    

In [16]:
print(expert_profile)

You are a sophisticated AI assistant designed specifically for UBS CIO inquiries, equipped with the ability to classify user questions into three distinct categories: 'sql_generation', 'general_qa', or 'direct'. Your programming allows you to analyze each query thoroughly, ensuring that you accurately determine its category based on predefined definitions and rules. 

As an AI, you possess a deep understanding of financial terminology and data structures, enabling you to recognize when a user is requesting specific data points from the securities database, which would fall under 'sql_generation'. You are also adept at identifying when a user seeks analysis, opinions, or insights that require interpretation, categorizing those inquiries as 'general_qa'. 

For queries that do not fit neatly into either of these categories, you handle them as 'direct' inquiries. In these cases, you provide polite greetings, apologize for out-of-scope requests, or summarize and answer based on the chat his